# Cargando lo que ya teníamos

In [4]:
import pandas as pd
import csv

In [5]:
df = pd.read_csv('corpus.csv')

## Distribución de palabras

Para entender un poco más el problema veremos la distribución de las palabras. Además esto nos va a ayudar a evitar problemas en un futuro.

In [6]:
from collections import Counter
from itertools import chain

In [7]:
distribution_words = Counter(chain.from_iterable(df.message.map(lambda x: x.split(' '))))

In [8]:
df_distribution = pd.DataFrame(data={'word': list(distribution_words.keys()), 'counter': list(distribution_words.values())})

Es decir el vocabulario total es alrededor de:

In [9]:
df_distribution.count()

counter    101766
word       101766
dtype: int64

Por lo que las palabras menos vistas son

In [10]:
df_distribution.sort_values('counter').head(20)

,counter,word
50882,1,¿somos
56242,1,Ajjaajjaa
56241,1,siéntase
56240,1,esceptica
56239,1,Tambn
56238,1,cuchareando
56234,1,Rift
95882,1,telef.
95885,1,"""Casos"
56230,1,elimine!!


Pero las más vistas son:

In [11]:
df_distribution.sort_values('counter', ascending=False).head(20)

,counter,word
2244,24708,que
97407,24334,de
81804,19958,a
56515,18938,la
41661,17263,el
13296,14857,no
17074,13412,y
53512,12577,en
49124,11407,omitido>
38721,11407,<Archivo


Al querer hacer un gráfico de caja, nos dimos cuenta que la media en la distribución de palabras era 1...

In [12]:
df_distribution.counter.median()

1.0

Pues luego de ver esa que la media era 1, nos preguntamos...
   - ¿Cuántas palabras hay con una sóla aparición? Son útiles o no?

In [13]:
sum(df_distribution.counter == 1) / df_distribution.count()

counter    0.648979
word       0.648979
dtype: float64

Vemos que hay alrededor de un 65% de palabras que aparecen una sóla vez... esto puede causar que la **vectorización no sea tan representativa como queremos...**

Problemas que vimos:
   - Al ser mensajes instantaneos, por lo general las personas no se preocupan con la ortográfia ni la forma de escribir.
   - Muchos tipos de mismas palabras, por ejemplo jajaja = jaja, dale = daleee.
   - _Arhivo omitido_ como media no es útil.
   - Puntaciones son inútiles para el problema (no ayudan en nada). Ejemplo quiero = quiero'
   - Los emoticones son necesarios para este problema?

## Conclusión

Después de discutir un rato decidimos tomar estás decisiones para mejorar la distribución:
   - "Normalizar palabras más vistas" (jajajajajaa --> jaja)
   - Tratar las puntuaciones como palabras ("?", "!", "...") o directamente removerlas (quiero' --> quiero)
   - Pasar a minuscula toda palabra
   - Eliminar palabra que no aparezca más de 10 veces (soluciona errores de ortografía, links, etc)
   - Remover acentos
   - Pasar digito a la palabra NUM
   - Remover characteres repetidos, ejemplo: vaaaaamos por vamos.
   - __Hacer Stemm serviría? FIJARSE __

Veamos como sería la nueva distribución

In [14]:
from string import punctuation
import re
import unicodedata

In [15]:
STOP_WORDS = punctuation + '¡'

def dig2num(text):
    return re.sub('(\d+[\WxX]*[^ ]*)+', 'NUM', text)

def rm_tildes(text):
    return ''.join((c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn'))

def rm_media(text):
    return re.sub(r'<archivo omitido>|<media omitted>', 'MEDIA', text)

def rm_repeated(text):
    return re.sub(r'([aeourshmi])\1{2,}', r'\1', text)

def rm_ok(text):
    return re.sub('(ok)+[akis]*', 'ok', text)

def rm_sw(text):
    return re.sub('{1}{0}{1}'.format(STOP_WORDS, '[]'), '', text)

def rm_ja(text):
    return re.sub('(ja[ja(js)]*)+', 'ja', text)

def rm_si(text):
    return re.sub('[s]+[i]+', 'si', text)

def to_low(text):
    return text.lower()

def clean_text(text):
    return rm_media(rm_tildes(to_low(text)))

def clean_text_1(text):
    return dig2num(rm_si(rm_ok(rm_ja(rm_repeated(rm_sw(clean_text(text)))))))

In [16]:
distribution_words = Counter(chain.from_iterable(df.message.map(lambda x: clean_text_1(x).split())))

In [17]:
df_distribution = pd.DataFrame(data={'word': list(distribution_words.keys()), 'counter': list(distribution_words.values())})

In [18]:
new_df = df_distribution[df_distribution.counter > 10]

In [19]:
new_df.count()

counter    5504
word       5504
dtype: int64

In [20]:
new_df.sort_values('counter', ascending=False).head(20)

,counter,word
2318,29526,que
49590,25647,de
33086,22398,a
38407,22198,no
6010,20865,la
44465,19731,el
43295,19547,ja
18305,18234,y
47026,16426,NUM
11233,16066,MEDIA


In [23]:
new_df.sort_values('counter').head(20)

,counter,word
1,11,danita
30727,11,crew
31455,11,experto
31499,11,sonrisa
31641,11,necesiten
31962,11,rapipago
32059,11,maldicion
32129,11,rec
32165,11,hacemo
32309,11,extension


Las listas van cobrando sentido